 ## Import Python Libraries

In [2]:
# !pip install fastapi[all] uvicorn nest-asyncio python-multipart pillow pytesseract pdf2image

In [3]:
import pdfplumber
import pytesseract
from PIL import Image
import cv2
import os
import io
import re
import pytesseract
from PIL import Image
from pdf2image import convert_from_path
from fastapi import FastAPI, UploadFile, File, Form, Request
from fastapi.responses import HTMLResponse, JSONResponse
from fastapi.staticfiles import StaticFiles
import nest_asyncio
import uvicorn

In [4]:
# Example path (change based on your folder)
data_folder = r"C:\Users\yoga1st\Insurance Policy\data"

# List files in the folder
all_files = os.listdir(data_folder)
print("Files found:", all_files)


Files found: ['.ipynb_checkpoints', '1.jpg', 'Accident_report_form_glove_box_MS018.pdf', 'Car-Insurance-Policy-Example-With-Premiums.png', 'homeowners-claim-form.pdf', 'sample-auto-insurance-declarations-page_1_2x.jpg', 'vehicle_accident_report.pdf', '_declarationpage__.png']


In [5]:
pdf_path = os.path.join(data_folder, "vehicle_accident_report.pdf")  # Replace with actual filename

with pdfplumber.open(pdf_path) as pdf:
    first_page = pdf.pages[0]
    text = first_page.extract_text()

print("PDF Extracted Text:\n")
print(text)


PDF Extracted Text:

Vehicle –
Western
ACCIDENT REPORTING FORM
To be completed at the scene. (Important: Do not admit liability or discuss any settlement.)
If there are personal injuries or severe damage to the vehicle, call 911.
If vehicle is drivable and if it’s safe to do so, pull to the side of road away from traffic.
Put out beacons or flares if available.
If you have a camera, record the damages at the scene.
Complete a detailed description of the accident and record all relevant information on this form.
File this report to the Western Corporate Insurance office within 24 hours.
File a report with the London Police Reporting Centre, 1001 Brydges Street London, if necessary.
Date of Accident:_______________________________ Time of Accident: ________________
Location of Accident: ___________________________________________________________
Western Vehicle Involved: ______________________________________________________
Make/Model/Year of Western Vehicle: ___________________________

In [6]:
import pytesseract
from PIL import Image

# ✅ Set the path to your installed tesseract.exe file
pytesseract.pytesseract.tesseract_cmd = r"C:\Users\yoga1st\tesseract.exe"

# ✅ Load an image and extract text
image = Image.open(r"C:\Users\yoga1st\Insurance Policy\data\_declarationpage__.png")
text = pytesseract.image_to_string(image)

print(text)

Sample insurance declara’

Insurance Company Policy Number: 123-45-678

Policy Effective Date: Jan 12024

Summary
Named Insured and Address Policy Number

Jane Doe 123-45-678

5432, Joy Lane

City, California 00000

Driver(s) Listed Policy Period

Jane Doe Effective: 01/01/2024 12:01 AM
John Doe Expiration: 01/01/2025 12:01 AM

Vehicle Information

Vehicle # Year/Make/Model/VIN
1 2015 Honda Accord 4D
WP1AA1234567890

Coverage Information

Coverage Limits
Bodily injury '$250K each person/$500K each incident
Property damage '$100K each incident

Permissive user limit of liability Full

Medical coverage $5,000 each person
Uninsured motorist bodily injury $250 each person/$500K each incident



In [7]:
import re

def extract_fields_universal(text: str):
    data = {}

    # Normalize text a bit
    text = re.sub(r"’", "'", text)
    text = re.sub(r"\r\n|\r", "\n", text)
    text = re.sub(r"\n{2,}", "\n\n", text)

    def extract_money(value_str):
        value_str = value_str.replace(',', '').replace('$', '').upper().strip()
        multiplier = 1
        if 'K' in value_str:
            multiplier = 1000
            value_str = value_str.replace('K', '')
        elif 'M' in value_str:
            multiplier = 1000000
            value_str = value_str.replace('M', '')
        try:
            return float(value_str) * multiplier
        except:
            return value_str

    # POLICY INFO
    policy_patterns = r"Policy (?:Number|No\.?|ID|#|Reference)[:\-\s]*([\w\-]+)"
    match = re.search(policy_patterns, text, re.IGNORECASE)
    data['Policy Number'] = match.group(1).strip() if match else None

    eff_date_patterns = r"(?:Policy|Coverage|Effective) (?:Date|Start)[:\-\s]*([A-Za-z0-9,\/\-\s]+)"
    match = re.search(eff_date_patterns, text, re.IGNORECASE)
    data['Policy Effective Date'] = match.group(1).strip() if match else None

    exp_date_patterns = r"(?:Policy|Coverage|Expiration|Expiry|End) (?:Date)?[:\-\s]*([A-Za-z0-9,\/\-\s]+)"
    match = re.search(exp_date_patterns, text, re.IGNORECASE)
    data['Policy Expiration Date'] = match.group(1).strip() if match else None

    # INSURED PERSON
    insured_patterns = [
        r"Named Insured and Address(?:\n+)([\w\s,\.]+)(?:\n+)([\w\s,\.]+)(?:\n+)([\w\s,\.]+)?",
        r"Named Insured[:\-\s]*([^\n]+)",
        r"Insured[:\-\s]*([^\n]+)",
        r"Employee Name[:\-\s]*([^\n]+)"
    ]
    insured = None
    address = None
    for pattern in insured_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            if len(match.groups()) >= 3:
                insured = match.group(1).strip()
                address = " ".join(filter(None, [match.group(2), match.group(3)])).strip()
            else:
                insured = match.group(1).strip()
            break
    data['Named Insured'] = insured
    if address:
        data['Address'] = address

    # DRIVERS or INSUREDS with dates
    drivers = []
    driver_pattern = r"(\b[A-Z][a-z]+ [A-Z][a-z]+)\s+(?:Effective|Start|From)[:\-\s]*([\d\/]+(?: \d{1,2}:\d{2} [AP]M)?)\s*(?:Expiration|End|To|Until)[:\-\s]*([\d\/]+(?: \d{1,2}:\d{2} [AP]M)?)?"
    for match in re.finditer(driver_pattern, text, re.IGNORECASE):
        drivers.append({
            "name": match.group(1),
            "effective": match.group(2),
            "expiration": match.group(3) if match.group(3) else None
        })
    data['Drivers'] = drivers if drivers else None

    # VEHICLE INFORMATION (many synonyms)
    vehicle_patterns = [
        r"Vehicle #?.*?Year/Make/Model/VIN.*?\n(?:\d+\s+)?(\d{4})\s+([\w\s]+)\s+([\w\s]+)\s*\n([A-Z0-9]{8,17})",
        r"Auto Details[:\-\s]*\n(?:Year[:\-\s]*(\d{4}))?.*?Make[:\-\s]*([\w\s]+)?.*?Model[:\-\s]*([\w\s]+)?.*?VIN[:\-\s]*([A-Z0-9]{8,17})",
        r"Car Description[:\-\s]*\nYear[:\-\s]*(\d{4})\nMake[:\-\s]*([\w\s]+)\nModel[:\-\s]*([\w\s]+)\nVIN[:\-\s]*([A-Z0-9]{8,17})"
    ]
    vehicle = None
    for pattern in vehicle_patterns:
        match = re.search(pattern, text, re.IGNORECASE | re.DOTALL)
        if match:
            vehicle = {
                "year": match.group(1) if match.group(1) else None,
                "make": match.group(2).strip() if match.group(2) else None,
                "model": match.group(3).strip() if match.group(3) else None,
                "vin": match.group(4).strip() if match.group(4) else None
            }
            break
    data['Vehicle'] = vehicle

    # HOME INSURANCE - common fields
    home_patterns = {
        "Property Details": r"Property (?:Details|Description)[:\-\s]*([\s\S]{10,300})",
        "Dwelling Coverage": r"Dwelling Coverage[:\-\s]*\$?([\d,\.KMkm]+)",
        "Loss Payee": r"Loss Payee[:\-\s]*([^\n]+)",
        "Fire Protection": r"Fire Protection[:\-\s]*([^\n]+)"
    }
    home_info = {}
    for key, pattern in home_patterns.items():
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            val = match.group(1).strip()
            if 'Coverage' in key:
                val = extract_money(val)
            home_info[key] = val
    data['Home Insurance'] = home_info if home_info else None

    # MEDICAL INSURANCE - common fields
    medical_patterns = {
        "Medical Benefits": r"Medical Benefits[:\-\s]*\$?([\d,\.KMkm]+)",
        "Hospital Coverage": r"Hospital Coverage[:\-\s]*\$?([\d,\.KMkm]+)",
        "Doctor Visits": r"Doctor Visits[:\-\s]*\$?([\d,\.KMkm]+)",
        "Prescription Coverage": r"Prescription Coverage[:\-\s]*\$?([\d,\.KMkm]+)"
    }
    medical_info = {}
    for key, pattern in medical_patterns.items():
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            val = match.group(1).strip()
            val = extract_money(val)
            medical_info[key] = val
    data['Medical Insurance'] = medical_info if medical_info else None

    # COVERAGE INFORMATION (general + vehicle + home + medical combined)
    coverage_patterns = {
        'Bodily Injury': r"(?:Bodily injury|BI)\s*['\"]?([\$\d\,KkMm\s\/each person\/each incident]+)",
        'Property Damage': r"(?:Property damage|PD)\s*['\"]?([\$\d\,KkMm\s\/each incident]+)",
        'Medical Coverage': r"Medical coverage\s*([\$\d\,KkMm\s]+)",
        'Uninsured Motorist Bodily Injury': r"Uninsured motorist bodily injury\s*([\$\d\,KkMm\s\/each person\/each incident]+)",
        'Permissive User Limit': r"Permissive user limit of liability\s*([^\n]+)"
    }
    coverage_info = {}
    for key, pattern in coverage_patterns.items():
        m = re.search(pattern, text, re.IGNORECASE)
        if m:
            val = m.group(1).strip()
            if 'Coverage' not in key and 'Limit' not in key:
                val = extract_money(val)
            coverage_info[key] = val
    data['Coverage Information'] = coverage_info if coverage_info else None

    # CLAIM TYPE - many synonyms
    claim_type_patterns = [
        r"Claim Type[:\-\s]*([^\n]+)",
        r"Type of Claim[:\-\s]*([^\n]+)",
        r"Loss Type[:\-\s]*([^\n]+)",
        r"Coverage Type[:\-\s]*([^\n]+)"
    ]
    claim_type = None
    for pattern in claim_type_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            claim_type = match.group(1).strip()
            break
    data['Claim Type'] = claim_type

    # CLAIM AMOUNT - many synonyms
    claim_amount_patterns = [
        r"Claim Amount[:\-\s]*\$?([\d,\.KMkm]+)",
        r"Amount Claimed[:\-\s]*\$?([\d,\.KMkm]+)",
        r"Loss Amount[:\-\s]*\$?([\d,\.KMkm]+)",
        r"Amount Paid[:\-\s]*\$?([\d,\.KMkm]+)"
    ]
    claim_amount = None
    for pattern in claim_amount_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            claim_amount = extract_money(match.group(1))
            break
    # fallback largest money value
    if not claim_amount:
        money_matches = re.findall(r"\$[\d,\.]+[KkMm]?", text)
        if money_matches:
            claim_amount = max(money_matches, key=lambda x: extract_money(x))
            claim_amount = extract_money(claim_amount)
    data['Claim Amount'] = claim_amount

    # INCIDENT DESCRIPTION with multiple possible headings
    desc_patterns = [
        r"Description of What Occurred[:\-\s]*([\s\S]{20,500})",
        r"Incident Description[:\-\s]*([\s\S]{20,500})",
        r"Details of any Injuries[:\-\s]*([\s\S]{20,500})",
        r"Accident Description[:\-\s]*([\s\S]{20,500})",
        r"Loss Description[:\-\s]*([\s\S]{20,500})"
    ]
    description = None
    for pattern in desc_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            description = match.group(1).strip()
            description = re.sub(r'\n+', ' ', description)[:500]
            break
    data['Incident Description'] = description

    return data


In [8]:
import pandas as pd

def create_dataframe(extracted_data_list):
    df = pd.DataFrame(extracted_data_list)
    # Clean/convert data types if needed, e.g. amounts to float
    df["Claim Amount"] = df["Claim Amount"].str.replace(",", "").astype(float)
    return df


In [9]:
# from fastapi import FastAPI, UploadFile, File, HTTPException
# from fastapi.responses import JSONResponse, HTMLResponse
# import io
# from PIL import Image
# import pytesseract
# import nest_asyncio
# import uvicorn

# # Your extract_text_from_pdf and extract_fields functions should be defined above or imported

# app = FastAPI()

# @app.get("/", response_class=HTMLResponse)
# async def upload_form():
#     return """
#     <html>
#         <head>
#             <title>Insurance OCR Parser</title>
#         </head>
#         <body>
#             <h2>Upload Insurance Document</h2>
#             <form action="/upload/" enctype="multipart/form-data" method="post">
#                 <input type="file" name="file" required><br><br>
#                 <button type="submit">Convert to JSON</button>
#             </form>
#         </body>
#     </html>
#     """

# @app.post("/upload/")
# async def upload_and_parse(file: UploadFile = File(...)):
#     try:
#         contents = await file.read()
        
#         # Save and extract text
#         if file.filename.lower().endswith(".pdf"):
#             with open("temp.pdf", "wb") as f:
#                 f.write(contents)
#             text = extract_text_from_pdf("temp.pdf")
#         else:
#             image = Image.open(io.BytesIO(contents))
#             text = pytesseract.image_to_string(image)
        
#         # Extract structured data
#         data = extract_fields_universal(text)
#         return JSONResponse(content=data)
    
#     except Exception as e:
#         # Return detailed error as HTTP 500
#         raise HTTPException(status_code=500, detail=f"Error processing file: {str(e)}")

# # For running inside Jupyter Notebook
# nest_asyncio.apply()
# uvicorn.run(app, host="127.0.0.1", port=8000)


In [ ]:
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import JSONResponse, HTMLResponse, FileResponse
import io
from PIL import Image
import pytesseract
import nest_asyncio
import uvicorn
import json

app = FastAPI()

@app.get("/", response_class=HTMLResponse)
async def upload_form():
    return """
    <html>
        <head>
            <title>Insurance Document Parser by Jharana</title>
            <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css">
            <style>
                body {
                    font-family: Arial, sans-serif;
                    background-color: #f4f4f4;
                    display: flex;
                    justify-content: center;
                    align-items: center;
                    height: 100vh;
                    margin: 0;
                }
                .container {
                    background-color: #ffffff;
                    padding: 40px;
                    border-radius: 15px;
                    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
                    text-align: center;
                    width: 100%;
                    max-width: 500px;
                }
                h2 {
                    color: #FF0000;
                }
                input[type="file"] {
                    margin: 20px 0;
                }
                .button {
                    background-color: #FF0000;
                    color: white;
                    padding: 12px 25px;
                    border: none;
                    border-radius: 8px;
                    font-size: 16px;
                    cursor: pointer;
                }
                .button:hover {
                    background-color: #cc0000;
                }
                .icon {
                    font-size: 30px;
                    color: #FF0000;
                    margin-bottom: 10px;
                }
            </style>
        </head>
        <body>
            <div class="container">
                <div class="icon"><i class="fas fa-file-upload"></i></div>
                <h1><i class="fas fa-file-medical-alt"></i> Insurance Document Parser by Jharana</h1>
                <h2>Upload Insurance Document</h2>
                <form action="/upload/" enctype="multipart/form-data" method="post">
                    <input type="file" name="file" required><br>
                    <button class="button" type="submit"><i class="fas fa-file-alt"></i> Convert to JSON</button>
                </form>
            </div>
        </body>
    </html>
    """

@app.post("/upload/", response_class=HTMLResponse)
async def upload_and_parse(file: UploadFile = File(...)):
    try:
        contents = await file.read()

        # Extract text
        if file.filename.lower().endswith(".pdf"):
            with open("temp.pdf", "wb") as f:
                f.write(contents)
            text = extract_text_from_pdf("temp.pdf")  # Your custom function
        else:
            image = Image.open(io.BytesIO(contents))
            text = pytesseract.image_to_string(image)

        # Extract fields
        data = extract_fields_universal(text)  # Your custom function

        with open("parsed_output.json", "w") as f:
            json.dump(data, f, indent=4)

        pretty_json = json.dumps(data, indent=4).replace("\n", "<br>").replace(" ", "&nbsp;")

        return f"""
        <html>
            <head>
                <title>Result - Insurance Parser</title>
                <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css">
                <style>
                    body {{
                        font-family: Arial, sans-serif;
                        background-color: #f9f9f9;
                        display: flex;
                        justify-content: center;
                        align-items: center;
                        height: 100vh;
                        margin: 0;
                    }}
                    .result-box {{
                        background-color: #ffffff;
                        padding: 30px;
                        border-radius: 15px;
                        box-shadow: 0 4px 8px rgba(0,0,0,0.2);
                        text-align: center;
                        width: 100%;
                        max-width: 700px;
                    }}
                    .json-box {{
                        text-align: left;
                        font-family: monospace;
                        background-color: #f0f0f0;
                        padding: 15px;
                        margin: 20px 0;
                        border-radius: 10px;
                        max-height: 300px;
                        overflow-y: auto;
                    }}
                    .button {{
                        background-color: #FF0000;
                        color: white;
                        padding: 12px 25px;
                        border: none;
                        border-radius: 8px;
                        font-size: 16px;
                        cursor: pointer;
                        text-decoration: none;
                    }}
                    .button:hover {{
                        background-color: #cc0000;
                    }}
                    .icon {{
                        font-size: 30px;
                        color: #FF0000;
                        margin-bottom: 10px;
                    }}
                </style>
            </head>
            <body>
                <div class="result-box">
                    <div class="icon"><i class="fas fa-file-alt"></i></div>
                    <h2>Parsed JSON Result</h2>
                    <div class="json-box">{pretty_json}</div>
                    <a href="/download/" class="button"><i class="fas fa-download"></i> Download JSON</a>
                </div>
            </body>
        </html>
        """

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error processing file: {str(e)}")

from fastapi.responses import JSONResponse

@app.get("/download/")
async def download_result():
    with open("parsed_output.json", "r") as f:
        data = json.load(f)
    return JSONResponse(content=data)

# For notebooks
nest_asyncio.apply()
uvicorn.run(app, host="127.0.0.1", port=8000)


INFO:     Started server process [10592]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:53732 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:53736 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:53737 - "POST /upload/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:53738 - "GET /download/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:53739 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:53740 - "GET /download/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:53741 - "POST /upload/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:53741 - "GET /download/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:53742 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:53746 - "POST /upload/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:53794 - "POST /upload/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:53795 - "POST /upload/ HTTP/1.1" 200 OK
